In [ ]:
import pandas as pd
import pywhatkit as wpp
from datetime import datetime, timedelta
import dataframe_image as dfi
import os

#SetOptions para ver los dataframes
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth',False)

In [ ]:
def dias_alertas():
    hoy = datetime.now()
    dias_semana = {'lunes': 0, 'martes': 1, 'miércoles': 2, 'jueves': 3, 'viernes': 4, 'sábado': 5, 'domingo': 6}

    # Encuentra el último martes y jueves
    last_tuesday = hoy - timedelta(days=(hoy.weekday() - dias_semana['martes']) % 7)
    last_thursday = hoy - timedelta(days=(hoy.weekday() - dias_semana['jueves']) % 7)
    
    # Devuelve la fecha más reciente entre el último martes y jueves
    return max(last_tuesday, last_thursday).strftime('%d')

In [ ]:
def clean_alertas(df):
    # Adios a los registros. Filtro solo Cadereyta
    # df = df.drop(columns=['Registro 1', 'Registro 2', 'Registro 3', 'Registro 4', 'Registro 5', 'Subproyecto'])
    df = df[df['Centro Enlace'].isin(['CADEREYTA', 'CE CADEREYTA', 'CC CADEREYTA'])]
    
    # Juntar las materias
    # inactivos_agrup = df.groupby(["Matricula"])["Grupo"].agg(lambda x: ", ".join(x)).reset_index().rename(columns={'Grupo':'Materias'})
    df['Materias'] = df.groupby(["Matricula", 'Estatus'])["Grupo"].transform(lambda x: ', '.join(x)) #.agg(lambda x: ", ".join(x)).reset_index().rename(columns={'Grupo':'Materias'})
    
    # df.shape #Materias Agrupadas en la última colunmna
    
    # Reacomodo de las materias
    df["Grupo"]  = df["Materias"]
    df.drop(columns = ["Materias", 'Modelo'], inplace=True)
    df = df.drop_duplicates(subset= ['Nombre', 'Estatus'])
    df.shape #sin duplicados 
    return df

In [ ]:
def clean_database(db):
    #Renombrar el nombre de la columna NOMBRE
    db.rename(columns = {'NOMBRE                                                                                                                                      ' : "NOMBRE", 'MATRICULA' : "Matricula"}, inplace= True)

    # Columnas importantes
    db = db[['NOMBRE', 'Matricula', 'CE CANALIZADO', 'CARRERA', 'MODELO EDUCATIVO', f'M{datetime.now().month}', 'Correo', 'Teléfono', 'Teléfono2', 'Teléfono3', 'CURP', 'Agente ', 'Asesor Académico', 'Asesor Financiero ', 'Asesor Escolar']]

    # Filtro Cadereyta
    # db = db[db['CE CANALIZADO'] == 'CE CADEREYTA']
    cade = ('CADEREYTA', 'CE CADEREYTA', 'CC CADEREYTA')
    db = db[db['CE CANALIZADO'].str.contains('|'.join(cade), case = False, na = False)]
    return db

In [ ]:
def filtro_estatus(df):
    option = int(input('1.- Inactivos \n2.- Ausentes \n3.- Regular \n 9.- Out'))

    if option == 1:
        # Inactivos
        option = ('Inactivo',)
    elif option == 2:
        # Ausente - Posible Reprobado
        option = ('Ausente', 'Posible Reprobado')
    elif option == 3:
        # Regular
        option = ('Regular',)
    return df[df['Estatus'].isin(option)]

In [ ]:
def save_infodata(df, long):
    df2 = pd.DataFrame(df, columns=['Matricula'])
    if len(df) == 0:
        df2.index = df2.index+1
        df2.loc[0] = datetime.now().strftime('%d/%m/%Y')
        df2 = df2.sort_index()
    else:
        df2.iloc[long+1:] = df2.iloc[long:-1].values
        df2.loc[long] = datetime.now().strftime('%d/%m/%Y')

    df2.to_csv('infodata/' + 'alertas_contactados.csv', index=False)

In [ ]:
def send_df(df, value):
    pd.set_option('display.max_colwidth',False)
    pd.set_option('display.width', 1000)
    route = 'Alertas{}.png'.format(value)
    
    # Filtrar filas donde cualquiera de las columnas contenga el valor
    df_final = df[df.apply(lambda row: row.astype(str).str.contains(value, case=False).any(), axis=1)]
    
    styled_df = df_final.style.set_properties(**{
    'white-space': 'pre-wrap',  # Permite el ajuste del texto
    'width': '200px',           # Ajustar el ancho de las columnas
    'max-width': '200px'
})
    display(df_final)
    if int(input('Enviar: 1')) == True:
        dfi.export(styled_df,route)
        with open('Infodata/IDGroup.txt', 'r') as f:
            for line in f:
                id_group = line.strip().split()[0] 
        wpp.sendwhats_image( id_group, route, route, tab_close=True)
        print('Guardada') if int(input('Guardar : 1')) == True else  os.remove(route)

Lectura de Excel

In [ ]:
path = 'Data/'

try:
    database = pd.read_csv(path + "BASE CNCI.csv") 
except FileNotFoundError or FileExistsError: 
    %run DatabaseCNCI.ipynb
    print('Database CNCI.csv Created')
    database = pd.read_csv(path + "BASE CNCI.csv")
    
try: 
    alertas = pd.read_csv(path + "Alertas.csv", index_col= 0)
except FileNotFoundError or FileExistsError:
    dict_mes={1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'}
    alertas = pd.read_excel(path + 'Main/Alertas tempranas {} de {} 2024.xlsx'.format(dias_alertas(), dict_mes[datetime.now().month]), sheet_name='General')
    clean_alertas(alertas).to_csv(path + 'Alertas.csv')
    print('Alertas csv Created')
    alertas = pd.read_csv(path + "Alertas.csv", index_col= 0)

In [ ]:
database = clean_database(database)
alertas = filtro_estatus(alertas)

In [ ]:
# Merge nuestros df, con el fin de no llamar a dos df en el for.
alertas_merged = pd.merge(alertas, database[['Matricula', 'MODELO EDUCATIVO','Teléfono', 'Teléfono2',]] , on = 'Matricula')

Archivos de utilidad

In [ ]:
imagen = 'img/Perseverancia.jpeg'
mencion_stat = ''
path2 = 'infodata/'

#Inicializar la infodata
matric_error = pd.read_csv(path2 + 'matriculas_error.csv')
matric_error = matric_error.dropna().values.flatten().tolist()
try:    
    matric_yacontactadas = pd.read_csv(path2+ 'alertas_contactados.csv') 
    matric_yacontactadas = matric_yacontactadas.dropna().values.flatten().tolist() 
except FileNotFoundError:
    matric_yacontactadas = list()
long = len(matric_yacontactadas)

Ciclo de mensajes

In [ ]:
for index, row in alertas_merged.iterrows():
    alumno = row['Nombre']
    matricula = row['Matricula']
    materias = row['Grupo']
    telefono = row['Teléfono']
    status = row['Estatus']
    mod = row['Modulo']
    modelo = row['MODELO EDUCATIVO']

    if not pd.isna(telefono):
        telefono = '+52 ' + str(telefono)
    if matricula in matric_error:
        if not pd.isna(row['Teléfono2']):
            telefono2 = '+52 ' + str(row['Teléfono2'])
            telefono = telefono2
            print('Matricula con error: {}, Alumno: {}'.format(matricula, alumno))
    if (mod == 4 or mod == 0) and modelo in ['FLEX', 'LIVE']:
        mencion_stat = 'estas en tu *Módulo de Descanso*, donde podrás descansar en caso de no tener Extraordinarios, solamente no olvides tu materia de {}'.format(materias)
    else:
        if status == ('Inactivo'):
            mencion_stat = '*{}*. Ya que veo que no has ingresado hace tiempo. \nA qué se debe esto? Has tenido algún problema con la _Plataforma Blackboard_? \nEn caso de que no hayas entregado tu *{}*, recuerda que aún la puedes entregar solo que con una calificación menor.'.format(materias, 'Actividad 1')
        elif status == ('Ausente'):
            mencion_stat = '*{}* Cómo te está yendo con las actividades? \nAdemás mencionarte que en caso de que no hayas entregado tu *{}* aún la puedes entregar solo que con una calificación menor. \nHas tenido algún problema con la _Plataforma Blackboard_?'.format(materias, 'Actividad 1')
        elif status == ('Posible Reprobado'):
            mencion_stat = '*{}* Cómo te está yendo con las actividades? \nAdemás mencionarte que en caso de que no hayas entregado tu *{}* aún la puedes entregar solo que con una calificación menor. \nPor último quería preguntarte si has tenido algún problema con la Plataforma?'.format(materias, 'Actividad 1')
        elif status == ('Regular'):
            mencion_stat = '*{}*. Llevandola(s) de una manera muy activa. Mencionar que tu Calificación #1 ya debe de estar en _Blackboard_. Has logrado visualizarla?\n\nMuchas felicidades y sigue así, quería preguntarte cómo te organizas para ir al corriente con las Actividades? Trabajas o solo Estudias?'.format(materias)
        
        
    if matricula in matric_yacontactadas:
        print('Matricula antes contactada: {}, Alumno: {} '.format(matricula, alumno))
    else:
        mensaje = 'Buen día para tí {} {}, te mando este mensaje para hablar sobre tus materias {} \n\nRecuerda que en caso de dudas puedes contactarme para así apoyarte, ten un buen día, quedo al pendiente de tu respuesta \n\n_Román Rodríguez Tutor Presencial_.'.format(alumno, matricula, mencion_stat)
        print('Enviado a: {} | {}'.format(matricula, alumno))
        print('--------------------')
        matric_yacontactadas.append(matricula)
        wpp.sendwhats_image(telefono, imagen, mensaje, 15, True)

save_infodata(matric_yacontactadas, long)

In [ ]:
def nomatch_mat():
    matching_rows = alertas[~alertas['Matricula'].isin(database['Matricula'])]
    return matching_rows
nomatch_mat()

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Lee la imagen
# print(alumno, resultado)
figure = Image.open(imagen)
# Muestra la imagen
plt.imshow(figure)
plt.axis('off')  # Desactiva los ejes
plt.show()